In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf


2023-04-12 15:43:00.255187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 1
EPOCHS = 50

In [4]:
df = pd.read_excel('height_and_pose.xlsx')
df

,Depthmap Image,Height(cm),Pose
0,1d5fee00-ada4-11eb-a80b-3f50af21830f,76.7,"[{'bbox_coordinates': [(158.91087, 92.53168), ..."
1,1eddf4c0-ad9f-11eb-a80b-ef4adba049aa,66.4,"[{'bbox_coordinates': [(131.00702, 68.115364),..."
2,38e29bc0-8fc0-11eb-b5f2-0742da91b282,63.2,"[{'bbox_coordinates': [(116.070114, 37.567608)..."
3,6cd5d480-8b8b-11eb-b152-f7b115384fa9,87.1,"[{'bbox_coordinates': [(145.24242, 26.49348), ..."
4,76ed6fc0-8e1d-11eb-ad70-7be1a1219834,59.1,"[{'bbox_coordinates': [(150.17046, 65.00379), ..."
5,8d23fa60-9201-11eb-b3e6-cbfb48c6a300,86.4,"[{'bbox_coordinates': [(146.97789, 37.342777),..."
6,9dc3eba0-8d8d-11eb-8d3f-4776be867c61,100.6,"[{'bbox_coordinates': [(135.09724, 42.205784),..."
7,9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4,106.1,"[{'bbox_coordinates': [(167.52087, 52.946323),..."
8,d3757800-8dff-11eb-a328-472d4f8db277,99.9,"[{'bbox_coordinates': [(92.025406, 19.148962),..."


In [5]:
from PIL import Image
depthmaps = []
for depthmap_name in df['Depthmap Image']:
    depthmap = Image.open('depthmap/' + depthmap_name + '.jpg')
    depthmap = np.array(depthmap.resize((256, 256)))
    depthmap = depthmap.reshape(256, 256, 1)
    depthmaps.append(depthmap)

In [6]:
depthmaps

[array([[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  1]],
 
        [[  0],
         [  0],
         [  0],
         ...,
         [ 67],
         [ 59],
         [  2]],
 
        [[  0],
         [  0],
         [  0],
         ...,
         [ 87],
         [ 76],
         [  4]],
 
        ...,
 
        [[  0],
         [ 17],
         [ 43],
         ...,
         [104],
         [ 90],
         [  4]],
 
        [[  0],
         [ 93],
         [239],
         ...,
         [  9],
         [  8],
         [  1]],
 
        [[  0],
         [ 24],
         [ 62],
         ...,
         [  0],
         [  0],
         [  2]]], dtype=uint8),
 array([[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  2]],
 
        [[  0],
         [  0],
         [  0],
         ...,
         [ 58],
         [ 51],
         [  3]],
 
        [[  0],
         [  0],
         [  0],
         ...,
      

New feature from the image mentioned in Depthmap Images column was created using PIL

In [7]:
X = np.array(depthmaps)
y = np.array(df['Height(cm)'])
# Since there is so less data train-dev-test split wasn't performed.

In [8]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [9]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

2023-04-12 15:43:03.513527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


As dataset is unstructed, I am using Neural Networks here. 

In [10]:
model = tf.keras.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [11]:
model.compile(loss='mean_squared_error', optimizer='adam')


In [12]:
X.shape

(9, 256, 256, 1)

In [13]:
y.shape

(9,)

In [14]:
model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE)


Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 7132.5986
Epoch 2/50
1/1 [==============================] - 0s 373ms/step - loss: 7130.9683
Epoch 3/50
1/1 [==============================] - 0s 360ms/step - loss: 7111.2832
Epoch 4/50
1/1 [==============================] - 0s 361ms/step - loss: 7039.3701
Epoch 5/50
1/1 [==============================] - 0s 348ms/step - loss: 6860.7710
Epoch 6/50
1/1 [==============================] - 0s 344ms/step - loss: 6443.7144
Epoch 7/50
1/1 [==============================] - 0s 345ms/step - loss: 5543.1152
Epoch 8/50
1/1 [==============================] - 0s 347ms/step - loss: 3822.0400
Epoch 9/50
1/1 [==============================] - 0s 341ms/step - loss: 1273.8706
Epoch 10/50
1/1 [==============================] - 0s 346ms/step - loss: 456.7959
Epoch 11/50
1/1 [==============================] - 0s 342ms/step - loss: 2094.8787
Epoch 12/50
1/1 [==============================] - 0s 341ms/step - loss: 350.8084
Epoch 13/50
1/1 [=

In [15]:
scores = model.evaluate(X, y)


1/1 [==============================] - 0s 222ms/step - loss: 57.8974


In [16]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [17]:
X[0].shape

(256, 256, 1)

In [18]:
X[0]

array([[[  0],
        [  0],
        [  0],
        ...,
        [  0],
        [  0],
        [  1]],

       [[  0],
        [  0],
        [  0],
        ...,
        [ 67],
        [ 59],
        [  2]],

       [[  0],
        [  0],
        [  0],
        ...,
        [ 87],
        [ 76],
        [  4]],

       ...,

       [[  0],
        [ 17],
        [ 43],
        ...,
        [104],
        [ 90],
        [  4]],

       [[  0],
        [ 93],
        [239],
        ...,
        [  9],
        [  8],
        [  1]],

       [[  0],
        [ 24],
        [ 62],
        ...,
        [  0],
        [  0],
        [  2]]], dtype=uint8)

In [20]:
model.predict(X)[0]


1/1 [==============================] - 0s 122ms/step


array([71.99635], dtype=float32)

In [21]:
y[0]

76.7